In [1]:
import json
from pathlib import Path

### Right-click on the file and select option to Copy Path

In [2]:
path = input("Path to file in bucket:")

Path to file in bucket: local-data/map.geojson


In [3]:
home = Path.home()
f_path = f'{home}/{path}'

In [4]:
try:
    if not Path(f_path).exists:
        print('FILE DOES NOT EXIST!')
    else:
        if Path(f_path).suffix == ".geojson":
            print('FILE EXISTS AND IS COMPATIBLE')
        else:
            print('FILE EXISTS BUT IS INCOMPATIBLE')
except CRSError:
    print('error', OSError)

FILE EXISTS AND IS COMPATIBLE


### If compatible, open file & create GeoJSON layer

In [5]:
from ipyleaflet import GeoJSON

with open(f_path, 'r') as f:
    data = json.load(f)

geojson = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
)
# print(json.dumps(geojson.data, indent=4))

### Calculate bounds (data extent) from all features

In [6]:
xcoords = []
ycoords = []
for f in data['features']:
    geom = f['geometry']
    for coord in geom['coordinates']:
        if type(coord) == float:  # then its a point feature
            xcoords.append(geom['coordinates'][0])
            ycoords.append(geom['coordinates'][1])
        elif type(coord) == list:
            for c in coord:
                if type(c) == float:  # then its a linestring feature
                    xcoords.append(coord[0])
                    ycoords.append(coord[1])
                elif type(c) == list:  # then its a polygon feature
                    xcoords.append(c[0])
                    ycoords.append(c[1])
bounds = [
    [min(ycoords), min(xcoords)],
    [max(ycoords), max(xcoords)]
]

### Add geojson to `stac_ipyleaflet` map

In [7]:
import stac_ipyleaflet

m = stac_ipyleaflet.StacIpyleaflet()
m.add_layer(geojson)
m.fit_bounds(bounds)
m

Output()

StacIpyleaflet(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…